# SPIDS Web Dashboard Tutorial

Learn how to use the interactive web dashboard for real-time training monitoring and multi-experiment comparison.

This tutorial covers:
1. Launching the dashboard
2. Monitoring training in real-time
3. Comparing multiple experiments
4. Interactive visualizations
5. Configuration comparison

**Estimated time**: 20-25 minutes

**Requirements**: 
- Dash and dash-bootstrap-components installed (included in `uv sync`)
- At least one completed experiment in `runs/` directory

## 1. Setup and Imports

In [ ]:
# Standard imports
import sys
import time
from pathlib import Path


sys.path.insert(0, "../..")

# SPIDS imports
from prism.web.launcher import DashboardLauncher
from prism.web.server import DashboardServer


# Check available experiments
runs_dir = Path("../../runs")
experiments = list(runs_dir.glob("*/checkpoint.pt"))

print(f"✓ Found {len(experiments)} experiments in runs/")
if len(experiments) > 0:
    print("\nAvailable experiments:")
    for exp in experiments[:5]:  # Show first 5
        print(f"  - {exp.parent.name}")
    if len(experiments) > 5:
        print(f"  ... and {len(experiments) - 5} more")
else:
    print("\n⚠️  No experiments found. Run a training first:")
    print("     cd ../.. && uv run python main.py --obj_name europa --n_samples 64 --max_epochs 10")

## 2. Dashboard Features Overview

The SPIDS dashboard provides:

### Real-Time Monitoring
- Live training progress updates
- Interactive Plotly charts (zoom, pan, hover)
- Current epoch, loss, SSIM, PSNR metrics
- Reconstruction preview vs ground truth

### Multi-Experiment Comparison
- Select and compare up to 4 experiments
- Side-by-side visualization
- Training curves overlay
- Configuration differences

### Interactive Visualizations
- K-space coverage heatmaps
- Sampling pattern visualization
- Synchronized zoom/pan
- Export plots as PNG

## 3. Launching the Dashboard

### Method 1: From Command Line (Recommended)

The simplest way to launch the dashboard:

```bash
cd ../..
spids dashboard --port 8050
```

Then open your browser to: http://localhost:8050

### Method 2: During Training

Launch dashboard alongside training:

```bash
cd ../..
python main.py --obj_name europa --n_samples 100 --dashboard
```

### Method 3: Programmatically (This Notebook)

Launch from Python code:

In [ ]:
# Launch dashboard in background
launcher = DashboardLauncher(port=8050)

print("Starting dashboard...")
launcher.start()

print("\n" + "=" * 60)
print("Dashboard is running!")
print("=" * 60)
print("\nOpen in your browser: http://localhost:8050")
print("\nPress Ctrl+C in the terminal to stop the dashboard.")
print("\nTo stop from this notebook, run: launcher.stop()")

## 4. Dashboard User Interface

### Main Components:

1. **Sidebar (Left)**
   - Experiment selector (dropdown with multi-select)
   - Refresh rate slider (1-10 seconds)
   - Quick actions

2. **Main Content (Right)**
   - Tabs: Training Metrics | Reconstructions | K-Space | Configuration
   - Interactive plots
   - Real-time updates

3. **Navigation Bar (Top)**
   - SPIDS branding
   - Current status indicator

### Navigation Tips:
- **Select experiments**: Use dropdown to choose 1-4 experiments
- **Zoom**: Click and drag on plots
- **Pan**: Hold Shift and drag
- **Hover**: Mouse over data points for details
- **Reset view**: Double-click on plot

## 5. Exploring Experiment Data

Let's load and explore experiment data programmatically:

In [ ]:
# Initialize dashboard server
server = DashboardServer(runs_dir=Path("../../runs"))

# Scan for experiments
available_exps = server.scan_experiments()

print(f"Found {len(available_exps)} experiments:\n")
for exp in available_exps[:5]:
    print(f"ID: {exp['id']}")
    print(f"  Path: {exp['path']}")
    print(f"  Last modified: {time.ctime(exp['last_modified'])}")
    print()

In [ ]:
# Load data for a specific experiment
if len(available_exps) > 0:
    exp_id = available_exps[0]["id"]
    exp_data = server.load_experiment_data(exp_id)

    print(f"Loaded experiment: {exp_id}\n")
    print("Metrics:")
    for key, value in exp_data["metrics"].items():
        print(f"  {key}: {value}")

    print("\nConfiguration highlights:")
    config = exp_data["config"]
    if "obj_name" in config:
        print(f"  Object: {config['obj_name']}")
    if "n_samples" in config:
        print(f"  Samples: {config['n_samples']}")
    if "pattern_type" in config:
        print(f"  Pattern: {config['pattern_type']}")
else:
    print("No experiments available to load.")

## 6. Multi-Experiment Comparison Workflow

### In the Dashboard:

1. **Select experiments**: In the sidebar dropdown, select 2-4 experiments
2. **View Training tab**: See overlaid training curves
3. **View Reconstructions tab**: Side-by-side comparison
4. **View Configuration tab**: See parameter differences

### Key Questions to Answer:

- Which configuration achieved the best metrics?
- How does training time compare?
- What parameters differ between experiments?
- Which sampling pattern works best?
- How does convergence behavior differ?

### Example Comparison Scenarios:

**Scenario 1: Pattern Comparison**
- Select experiments with different sampling patterns (fermat, random, star)
- Compare final SSIM values
- Observe convergence speed

**Scenario 2: Sample Count Study**
- Select experiments with 50, 100, 150, 200 samples
- See quality vs efficiency tradeoff
- Identify optimal sample count

**Scenario 3: Hyperparameter Tuning**
- Compare learning rates (0.001, 0.005, 0.01)
- Compare batch sizes
- Compare optimizer choices

## 7. Interactive Features Demo

### Training Curves
- **Zoom**: Focus on convergence region
- **Toggle lines**: Click legend entries to show/hide
- **Smoothing**: Adjust slider to smooth noisy curves
- **Export**: Camera icon to save as PNG

### Reconstructions
- **Colormap**: Change color scheme
- **Zoom**: Examine fine details
- **Toggle views**: Switch between magnitude/phase

### K-Space Coverage
- **Heatmap**: See sampling density
- **Radial profile**: Plot coverage vs frequency
- **Pattern overlay**: Show sampling positions

## 8. Real-Time Monitoring During Training

When training with `--dashboard` flag:

```bash
python main.py --obj_name europa --n_samples 100 --dashboard
```

The dashboard will:
1. Auto-refresh every 2 seconds (configurable)
2. Show current epoch and progress
3. Update training curves in real-time
4. Display latest reconstruction
5. Show ETA and elapsed time

### Real-Time Features:
- **Live metrics**: Loss, SSIM, PSNR updated continuously
- **Progress indicator**: Visual progress bar
- **Current reconstruction**: See latest output
- **Training speed**: Iterations per second
- **Memory usage**: GPU/CPU memory stats

## 9. Configuration Comparison

The Configuration tab shows:

### Parameter Diff Table
- **Green**: Parameters that improved results
- **Red**: Parameters that worsened results
- **Gray**: Neutral parameters

### Comparison Insights
- Which parameters have the most impact?
- Are there consistent patterns?
- What's the sensitivity to each parameter?

### Example Analysis:
```
Parameter       Exp1    Exp2    Exp3    Impact
─────────────────────────────────────────────
n_samples       50      100     150     HIGH
learning_rate   0.001   0.001   0.005   LOW
pattern         fermat  random  fermat  MEDIUM
```

## 10. Tips and Tricks

### Performance Tips:
1. **Limit experiments**: Compare max 4 at once for responsiveness
2. **Adjust refresh rate**: Slower rate = less CPU usage
3. **Close unused tabs**: Reduces rendering overhead

### Visualization Tips:
1. **Use log scale**: For loss curves with large dynamic range
2. **Normalize metrics**: Compare experiments at different scales
3. **Export plots**: Use camera icon for presentations

### Workflow Tips:
1. **Bookmark experiments**: Save interesting ones for later
2. **Take screenshots**: Document interesting findings
3. **Use multiple tabs**: Compare different aspects simultaneously

### Troubleshooting:
- **Dashboard not loading**: Check port 8050 is not in use
- **No data showing**: Verify experiments have checkpoints
- **Slow updates**: Reduce refresh rate or number of experiments
- **Memory issues**: Close other applications, limit experiments

## 11. Stopping the Dashboard

To stop the dashboard launched from this notebook:

In [ ]:
# Stop dashboard
try:
    launcher.stop()
    print("✓ Dashboard stopped successfully")
except Exception:
    print("⚠️  No dashboard running or already stopped")

## 12. Next Steps

### Explore More:
- Try comparing experiments with different patterns
- Monitor a long training run in real-time
- Export interesting plots for presentations
- Identify optimal hyperparameters

### Related Tools:
- **Compare CLI**: `spids compare runs/exp1 runs/exp2` - Terminal comparison
- **Inspect CLI**: `spids inspect runs/exp/checkpoint.pt` - Detailed inspection
- **Report Generation**: See Tutorial 5 for automated reports

### Advanced Usage:
- Custom dashboard layouts
- Remote dashboard access (SSH tunneling)
- Dashboard API for automation
- Integration with Jupyter notebooks

## Summary

You've learned:
- ✓ How to launch the SPIDS dashboard
- ✓ Real-time training monitoring
- ✓ Multi-experiment comparison
- ✓ Interactive visualizations
- ✓ Configuration analysis

**Next tutorial**: Tutorial 5: Automatic Report Generation

---

**Resources**:
- [Dashboard Documentation](../../docs/UI_UX_UPGRADE_PLAN.md#phase-2-interactive-web-dashboard)
- [API Reference](../../docs/user_guide.md)
- [GitHub Issues](https://github.com/username/SPIDS/issues)